In [1]:
import sys
sys.path.append("/home/q621464/Desktop/Thesis/code/decision-transformer-thesis")
sys.path.append("/home/q621464/Desktop/Thesis/code/decision-transformer-thesis/atari")

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import csv
import logging
# make deterministic
from atari.mingpt.utils import set_seed
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.utils.data import Dataset
from atari.mingpt.model_atari import GPT, GPTConfig
from atari.mingpt.trainer_atari import Trainer, TrainerConfig
from atari.mingpt.utils import sample
from collections import deque
import random
import torch
import pickle
import blosc
import argparse
from atari.create_dataset import create_dataset
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Apply dimensionality reduction here
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

2023-08-23 15:35:02.093547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 15:35:02.211425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/q621464/miniconda3/envs/decision-transformer-atari/lib/python3.7/site-packages/cv2/../../lib64::/home/q621464/.mujoco/mujoco200/bin
2023-08-23 15:35:02.211438: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-23 15:35:02.733665: W tensorflow/compiler/xla/stream_executor/platform/default/d

In [4]:
# set up logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [5]:
class Config():
    def __init__(self, seed=123, context_length=30, epochs=5, model_type='reward_conditioned', num_steps=500000, num_buffers=50, env='SmartClimate', batch_size=128, log_to_wandb=False, trajectories_per_buffer=10, train_data_dir='../atari/data-for-dt/smart-climate-train-trajectories-v9.pkl', val_data_dir='../atari/data-for-dt/smart-climate-val-trajectories-v9.pkl', test_data_dir='../atari/data-for-dt/smart-climate-test-trajectories-v9.pkl') -> None:
        self.seed = seed
        self.context_length = context_length
        self.epochs = epochs
        self.model_type = model_type
        self.num_steps =num_steps
        self.num_buffers = num_buffers
        self.env = env
        self.batch_size = batch_size
        self.log_to_wandb = log_to_wandb
        self.trajectories_per_buffer = trajectories_per_buffer
        self.train_data_dir = train_data_dir
        self.val_data_dir = val_data_dir
        self.test_data_dir = test_data_dir
        self.dim_reductor = None

In [6]:
args = Config(env='SmartClimate', epochs=30, seed=123, context_length=20, train_data_dir='../atari/data-for-dt/smart-climate-train-trajectories-v9.pkl')
set_seed(args.seed)

In [7]:
def reduce_dimensionality(obss):
    X_train = obss
    # Data normalizationd
    # scaler = StandardScaler()
    
    # scaler = MinMaxScaler(feature_range=(0, 1))
    
    # X_train_normalized = 2 * scaler.fit_transform(X_train) - 1
    
    X_train_normalized = obss

    # Initialize PCA with the desired number of components (e.g., None for all components)
    pca = PCA(n_components=None)

    # Fit and transform the training data using PCA
    X_train_pca = pca.fit_transform(X_train_normalized)

    # Get the explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_

    # Get the cumulative explained variance ratio
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    # Set a threshold for the cumulative explained variance ratio (e.g., 95% or 99%)
    threshold_variance = 0.85

    # Determine the number of components required to achieve the threshold
    n_components = np.argmax(cumulative_variance_ratio >= threshold_variance) + 1
    print(f"Number of selected components: {n_components}")

    # Reduce the dimensionality to the selected number of components
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_normalized)
    print(f"Shape of the reeduced dimensionality dataset: {X_train_pca.shape}")

    obss_reduced = X_train_pca
    return obss_reduced, pca
    # X_test_pca = pca.transform(X_test)

In [8]:
def create_dataset(data_dir, total_trajectories=8000):
    with open(data_dir, 'rb') as f:
        trajectories = pickle.load(f)[0:total_trajectories]
    obss = []
    actions = []
    returns = [0]
    done_idxs = []
    stepwise_returns = []    
    for traj in trajectories:
        obss += traj['observations'].tolist()
        actions += traj['actions'].tolist()
        stepwise_returns += traj['rewards'].tolist()
        done_idxs += [len(obss)]
        returns += [0]

    actions = np.array(actions)
    returns = np.array(returns)
    stepwise_returns = np.array(stepwise_returns)
    done_idxs = np.array(done_idxs)

    # -- create reward-to-go dataset
    start_index = 0
    rtg = np.zeros_like(stepwise_returns)
    for i in done_idxs:
        i = int(i)
        curr_traj_returns = stepwise_returns[start_index:i]
        for j in range(i-1, start_index-1, -1): # start from i-1
            rtg_j = curr_traj_returns[j-start_index:i-start_index]
            rtg[j] = sum(rtg_j)
        start_index = i
    print('max rtg is %d' % max(rtg))

    # -- create timestep dataset
    start_index = 0
    timesteps = np.zeros(len(actions)+1, dtype=int)
    print(f"total done idx: {len(done_idxs)}")
    for i in done_idxs:
        # print(f"done_idx: {i}")
        i = int(i)
        # print(f"start_idx: {start_index}, i: {i}")
        timesteps[start_index:i+1] = np.arange(i+1 - start_index)
        start_index = i+1
    print('max timestep is %d' % max(timesteps))
    
    # if apply_dim_reduction:
    #     obss, dim_reductor = reduce_dimensionality(obss)
    return obss, actions, returns, done_idxs, rtg, timesteps

In [9]:
class StateActionReturnDataset(Dataset):

    def __init__(self, data, block_size, actions, done_idxs, rtgs, timesteps, mean=0, std=1):
        self.block_size = block_size
        self.vocab_size = max(actions) + 1 # TODO: needs to be changed. Does it change dynamically based on the sampled data?
        self.data = data
        self.actions = actions
        self.done_idxs = done_idxs
        self.rtgs = rtgs
        self.timesteps = timesteps
        self.mean = mean
        self.std = std
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # print(f"Sampled idx: {idx}")
        block_size = self.block_size // 3
        states = torch.tensor(np.array(self.data[idx:idx+block_size]), dtype=torch.float32).reshape(block_size, -1) # 
        # (block_size, 4*84*84)
        
        # print(f"There are nan values in the dataloader's batch: {torch.isnan(states).any()}")
        # mean = torch.mean(states)
        # std = torch.std(states)
        # states = (states - self.mean) / self.std
        # print(f"mean: {mean}, std: {std} of the batch\n")
        # states = states / 255.
        # print(f"There are nan values in the dataloader's batch after normalization: {torch.isnan(states).any()}")
        actions = torch.tensor(self.actions[idx:idx+block_size], dtype=torch.long).unsqueeze(1) # (block_size, 1)
        rtgs = torch.tensor(self.rtgs[idx:idx+block_size], dtype=torch.float32).unsqueeze(1)
        timesteps = torch.tensor(self.timesteps[idx:idx+1], dtype=torch.int64).unsqueeze(1)
        # print(f"timesteps shape: {timesteps.shape}")

        return states, actions, rtgs, timesteps

In [10]:
# Create the train dataset first
obss, actions, returns, done_idxs, rtgs, timesteps = create_dataset(args.train_data_dir, total_trajectories=200)

# Create the validation dataset
obss_val, actions_val, returns_val, done_idxs_val, rtgs_val, timesteps_val = create_dataset(args.val_data_dir, total_trajectories=200)

# Create the validation dataset
obss_test, actions_test, returns_test, done_idxs_test, rtgs_test, timesteps_test = create_dataset(args.test_data_dir, total_trajectories=2000)

# Sanity check
# Are there any nan values in the obss
print(f"There are nan values in the obss: {np.isnan(np.array(obss)).any()}")

print("*" * len(args.env + "Environment"))
print(f"{args.env} Environment")
print("*" * len(args.env + "Environment"))
print(f"total obss: {len(obss)}\ntotal actions: {actions.shape}\ntotal returns: {returns.shape}\ntimesteps: {len(timesteps)}")

max rtg is 552
total done idx: 200
max timestep is 552
max rtg is 551
total done idx: 165
max timestep is 552
max rtg is 556
total done idx: 166
max timestep is 559
There are nan values in the obss: False
***********************
SmartClimate Environment
***********************
total obss: 50574
total actions: (50574,)
total returns: (201,)
timesteps: 50575


In [11]:
# used for input normalization
states = np.concatenate(obss, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
state_mean, state_std

(1.22437387541029, 5.311578543822653)

In [12]:
# # With PCA

# scaler = MinMaxScaler(feature_range=(0, 1))
# # Fit and transform the data to perform z-score normalization
# X_train_normalized = 2 * scaler.fit_transform(obss) - 1
# obss, dim_reductor = reduce_dimensionality(X_train_normalized)

# # scaler_val = MinMaxScaler(feature_range=(0, 1))
# obss_val_norm = 2 * scaler.transform(obss_val) - 1
# obss_val = dim_reductor.transform(obss_val_norm)

In [13]:
# # Without PCA

# scaler = MinMaxScaler(feature_range=(0, 1))
# # Fit and transform the data to perform z-score normalization
# obss = 2 * scaler.fit_transform(obss) - 1

# # scaler_val = MinMaxScaler(feature_range=(0, 1))
# obss_val = 2 * scaler.transform(obss_val) - 1

In [14]:

args.context_length = 20
train_dataset = StateActionReturnDataset(obss, args.context_length*3, actions, done_idxs, rtgs, timesteps)

val_dataset = StateActionReturnDataset(obss_val, args.context_length*3, actions_val, done_idxs_val, rtgs_val, timesteps_val)

test_dataset = StateActionReturnDataset(obss_test, args.context_length*3, actions_test, done_idxs_test, rtgs_test, timesteps_test)
# print(f"vocab size: {train_dataset.vocab_size}")

##### Best hyperparameters from hyperparameter tuning: 
1. Aimed at minimizing val_loss: {'learning_rate': 0.00019714606731026987, 'num_hidden_units': 40, 'context_length': 20, 'n_layer': 4}
2. Aimed at increasing val_accuracy: {'learning_rate': 0.0003999625589556598, 'num_hidden_units': 208, 'context_length': 10, 'n_layer': 8}

In [15]:
max_timestep = max(np.concatenate([timesteps, timesteps_val, timesteps_test]))
print(f"max_timestep: {max_timestep}")

max_timestep: 559


In [16]:
# if max(timesteps) > max(timesteps_val):
#     max_timestep = max(timesteps)
# else:
#     max_timestep = max(timesteps_val)


mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=3, n_head=1, n_embd=128, model_type=args.model_type, max_timestep=max_timestep, input_dim=len(obss[0]))
model = GPT(mconf)
model

GPT(
  (tok_emb): Embedding(25, 128)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU()
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln

In [17]:
# initialize a trainer instance and kick off training
args.epochs = 10
args.batch_size = 64
tconf = TrainerConfig(max_epochs=args.epochs, batch_size=args.batch_size, learning_rate=1e-4, lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*args.context_length*3, num_workers=4, seed=args.seed, model_type=args.model_type, env=args.env, max_timestep=max(timesteps))

In [18]:
len(obss[0])

16

In [19]:
print(f"Total train events: {len(train_dataset)}")
rtg = 100
max_ep_len = 100
trainer = Trainer(model, train_dataset, test_dataset, val_dataset, tconf, args.env, state_mean=state_mean, state_std=state_std, dim_reductor=None, scaler=None, rtg=rtg, max_ep_len=max_ep_len, num_eval_episodes=10)

# returns, predicted_actions, target_actions, epoch_losses, val_losses = trainer.train()
results = trainer.train()

print(f"Average reward achieved: {np.mean(results['epoch_returns']):.2f} with RTG: {rtg}")

Total train events: 50514


epoch 1 iter 789: train loss 0.21948. lr 9.341160e-05: 100%|██████████| 790/790 [02:00<00:00,  6.57it/s]
(idx, step): (68, 31) | True temperature: 17.0 | Predicted temperature: 24.0 | reward: 0.17:   0%|          | 0/100 [00:02<?, ?it/s] 

reward from this episode: 168.1128472222221 from 331 steps, in 100: 50.78937982544474


(idx, step): (97, 21) | True temperature: 18.5 | Predicted temperature: 23.5 | reward: 0.34:   0%|          | 0/100 [00:05<?, ?it/s] 

reward from this episode: 176.77777777777783 from 662 steps, in 100: 26.703591809331996


...

reward from this episode: 182.62847222222044 from 1997 steps, in 100: 9.145141323095666


(idx, step): (163, 104) | True temperature: 23.0 | Predicted temperature: 22.5 | reward: 0.92:   0%|          | 0/100 [00:14<?, ?it/s]


reward from this episode: 86.50173611111107 from 2101 steps, in 100: 4.117169733989104
total num_steps in the env: 12427, total rewards: 1017.1666666666646
target return: 100, eval return: 8, total steps: 2101


epoch 3 iter 533: train loss 0.22573. lr 5.857218e-05:  68%|██████▊   | 534/790 [01:26<00:41,  6.19it/s]


KeyboardInterrupt: 

#### Testing

In [ ]:
loss, accuracy, test_targets, test_preds = trainer.test_model()

In [ ]:
accuracy

In [ ]:
test_targets_merged = torch.cat(test_targets).flatten()
test_preds_merged = torch.cat(test_preds).flatten()

In [ ]:
((test_targets_merged == test_preds_merged).sum()) * 100. / len(test_targets_merged)

In [ ]:
sns.histplot(test_targets_merged)

In [ ]:
sns.histplot(test_preds_merged)

### Results investigation

In [ ]:
epoch_losses = results['epoch_losses']
val_losses = results['val_losses']
predicted_actions = results['predicted _actions']
target_actions = results['target_actions']
train_epoch_accuracies = results['train_accuracies']
val_accuracies = results['val_accuracies']
returns = results['epoch_returns']

In [ ]:
plt.plot(epoch_losses, label='train_loss')
plt.plot(val_losses, label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f"train and val loss for {len(train_dataset)} train and {len(val_dataset)} datapoints with batch size {args.batch_size}")
# plt.show()
# plt.savefig(f'Figures/train_and_val_loss_for_{len(train_dataset)}_datapoints.jpg')

In [ ]:
plt.plot(train_epoch_accuracies, label='train_acc')
plt.plot(val_accuracies, label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f"train and val acc for {len(train_dataset)} train and {len(val_dataset)} datapoints")
# plt.show()
# plt.savefig(f'Figures/train_and_val_loss_for_{len(train_dataset)}_datapoints.jpg')

In [ ]:
from atari.mingpt.envs.smart_climate_env import CustomActionSpace
action_map = CustomActionSpace().actions_map

train_actions_str = [str(action_map[action]) for action in actions]
predicted_actions_str = [str(action) for action in predicted_actions]
target_actions_str = [str(action) for action in target_actions]
val_actions_str = [str(action_map[action]) for action in actions_val]

In [ ]:
train_actions_unique, counts = np.unique(train_actions_str, return_counts=True)
train_action_count_dict = dict(zip(train_actions_unique, counts))

target_actions_unique, counts = np.unique(target_actions_str, return_counts=True)
target_action_count_dict = dict(zip(target_actions_unique, counts))

predicted_actions_unique, counts = np.unique(predicted_actions_str, return_counts=True)
predicted_action_count_dict = dict(zip(predicted_actions_unique, counts))

val_actions_unique, counts = np.unique(val_actions_str, return_counts=True)
val_action_count_dict = dict(zip(val_actions_unique, counts))

for action in np.arange(16.0, 28.5, 0.5):
    if str(action) not in predicted_action_count_dict.keys():
        predicted_action_count_dict[str(action)] = 0
    if str(action) not in train_action_count_dict.keys():
        train_action_count_dict[str(action)] = 0
    if str(action) not in target_action_count_dict.keys():
        target_action_count_dict[str(action)] = 0
    if str(action) not in val_action_count_dict.keys():
        val_action_count_dict[str(action)] = 0

sorted_keys = sorted(predicted_action_count_dict)
predicted_action_count_dict = {key: predicted_action_count_dict[key] for key in sorted_keys}
train_action_count_dict = {key: train_action_count_dict[key] for key in sorted_keys}
target_action_count_dict = {key: target_action_count_dict[key] for key in sorted_keys}
val_action_count_dict = {key: val_action_count_dict[key] for key in sorted_keys}

sorted(train_action_count_dict.keys()) == sorted(predicted_action_count_dict.keys())

In [ ]:
accuracy = accuracy_score(target_actions_str, predicted_actions_str)
print(f"Evaluation Accuracy: {accuracy * 100:.4f}")

In [ ]:
from sklearn.metrics import f1_score
macro_f1 = f1_score(target_actions_str, predicted_actions_str, average='macro')

# Calculate the micro F1 score
micro_f1 = f1_score(target_actions_str, predicted_actions_str, average='micro')

print("Macro F1 Score:", macro_f1)
print("Micro F1 Score:", micro_f1)

In [ ]:
classes = np.arange(16, 28.5, 0.5)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

for target, prediction in zip(target_actions, predicted_actions):
    if target == prediction:
        correct_pred[target] += 1
    total_pred[target] += 1
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    if total_pred[classname] > 0:
        accuracy = 100 * float(correct_count) / total_pred[classname]
    else:
        accuracy = 0
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
# Calculate and display the classification report for each class
class_names = sorted(np.unique(train_actions_str))
report = classification_report(target_actions_str, predicted_actions_str, target_names=class_names)
print("Classification Report:")
print(report)

In [ ]:
# Sample data for three datasets
data1 = train_action_count_dict.values()
data4 = val_action_count_dict.values()
data2 = target_action_count_dict.values()
data3 = predicted_action_count_dict.values()

# Define the x-axis labels for the bars
x_labels = train_action_count_dict.keys()

# Create a figure and three subplots side by side
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))

# Plot the bar plots on each subplot
bar_width = 1
x = np.arange(len(x_labels))

# Bar plot for Dataset 1
axes[0].bar(x, data1, bar_width, color='blue', label='train actions', edgecolor='black', alpha=0.5)
axes[0].set_xticks(x)
axes[0].set_xticklabels(x_labels, rotation=90)
axes[0].set_title('Target temperature from train dataset')

# Bar plot for Dataset 4
axes[1].bar(x, data4, bar_width, color='purple', label='predicted actions', edgecolor='black', alpha=0.5)
axes[1].set_xticks(x)
axes[1].set_xticklabels(x_labels, rotation=90)
axes[1].set_title('Target temperature from the the validation dataset')

# Bar plot for Dataset 2
axes[2].bar(x, data2, bar_width, color='green', label='target actions', edgecolor='black', alpha=0.5)
axes[2].set_xticks(x)
axes[2].set_xticklabels(x_labels, rotation=90)
axes[2].set_title('Target temperature from the test set')

# Bar plot for Dataset 3
axes[3].bar(x, data3, bar_width, color='red', label='predicted actions', edgecolor='black', alpha=0.5)
axes[3].set_xticks(x)
axes[3].set_xticklabels(x_labels, rotation=90)
axes[3].set_title('Target temperature from the predictions on the test set')



# Add labels and title to the overall figure
plt.suptitle('Bar Plots of Train, Eval and Predicted target temperatures')
plt.tight_layout()
plt.savefig('Figures/temp.jpg')
plt.show()


In [ ]:
import numpy as np

def compute_probabilities(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    total_counts = len(labels)
    probabilities = counts / total_counts
    return unique_labels, probabilities

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))


# Compute probabilities and unique labels for each list
labels1, probabilities1 = compute_probabilities(train_actions_str)
labels2, probabilities2 = compute_probabilities(val_actions_str)

# Union of unique labels from both lists
all_labels = np.union1d(labels1, labels2)

# Fill missing labels with zero probabilities
probabilities1_all = np.zeros_like(all_labels, dtype=float)
probabilities2_all = np.zeros_like(all_labels, dtype=float)
probabilities1_all[np.searchsorted(all_labels, labels1)] = probabilities1
probabilities2_all[np.searchsorted(all_labels, labels2)] = probabilities2

# Compute KL-divergence from list1 to list2
kl_divergence_1to2 = kl_divergence(probabilities1_all, probabilities2_all)

# Compute KL-divergence from list2 to list1
kl_divergence_2to1 = kl_divergence(probabilities2_all, probabilities1_all)

print("KL-Divergence from list1 to list2:", kl_divergence_1to2)
print("KL-Divergence from list2 to list1:", kl_divergence_2to1)


In [ ]:
train_actions = [float(action) for action in train_actions_str]
val_actions = [float(action) for action in val_actions_str]

print(np.mean(train_actions), np.mean(val_actions))
print(np.median(train_actions), np.median(val_actions))
print(np.std(train_actions), np.std(val_actions))

In [ ]:
# Build confusion matrix
unique_labels = np.unique(np.concatenate((target_actions_str, predicted_actions_str)))

# Build confusion matrix
cf_matrix = confusion_matrix(target_actions_str, predicted_actions_str, labels=unique_labels)

# Create a DataFrame from the confusion matrix for better visualization
df_cm = pd.DataFrame(cf_matrix, index=unique_labels, columns=unique_labels)

plt.figure(figsize = (16,10))
# plt.plot(df_cm)
sn.heatmap(df_cm, annot=True, cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f"Confusion matrix for {len(train_dataset)} train and {len(predicted_actions)} datapoints")
# plt.show()
plt.savefig(f'Figures/Confusion matrix for {len(train_dataset)} train and {len(predicted_actions)} datapoints.jpg')


In [ ]:
sns.lineplot(returns)
plt.title("Return using sampling from the test env")
plt.show()
# plt.savefig(f"Figures/return_with_sampling_500_train_traj.jpg")

In [ ]:
sns.histplot(sorted(train_actions_str))
plt.xlabel('Target Temperatures')
plt.title("Target Temperature distribution in the training set")
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.histplot(sorted(target_actions_str))
plt.xlabel('Target temperature')
plt.title("Target temperatures distribution in the evaluation set")
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.histplot(sorted(predicted_actions_str))
plt.xlabel('Target temperatures from the evaluation set')
plt.title("Predicted target temperatures distribution from the evaluation set")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# # Numbers of pairs of bars you want
# # N = 4

# # Data on X-axis

# # Specify the values of blue bars (height)
# # train_actions = list(train_action_count_dict.values())
# target_actions = list(target_action_count_dict.values())


# # Specify the values of orange bars (height)
# pred_actions = list(predicted_action_count_dict.values())


# print(len(target_actions), len(pred_actions))
# # Position of bars on x-axis
# ind = np.arange(16.0, 28.5, 0.5)
# # ind = np.arange(5)
# # print(ind)

# # Figure size
# plt.figure(figsize=(10,5))

# # Width of a bar 
# width = 0.2       

# # Plotting
# plt.bar(ind, target_actions , width, label='target action')
# plt.bar(ind + width, pred_actions, width, label='pred action')

# plt.xlabel('Actions')
# plt.ylabel('Count')
# plt.title('Action distribution comparison between target and predictions')

# # # xticks()
# # # First argument - A list of positions at which ticks should be placed
# # # Second argument -  A list of labels to place at the given locations
# # plt.xticks(ind+0.1, train_action_count_dict.keys())
# plt.xticks(ind+0.1, ind)

# # Finding the best position for legends and putting it
# plt.legend(loc='best')
# plt.xticks(rotation=90)
# plt.show()

### Hyperparameter Tuning

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# from your_pytorch_model import YourModel  # Import your PyTorch model

def objective(trial):
    # Sample hyperparameters to be tuned by Optuna
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    num_hidden_units = trial.suggest_int('num_hidden_units', 32, 256)
    
    # Load the val_dataset
    val_data_dir = "../atari/data-for-dt/smart-climate-val-trajectories-v2.pkl"
    # Create the dataset first
    obss, actions, returns, done_idxs, rtgs, timesteps = create_dataset(val_data_dir, total_trajectories=8000, apply_dim_reduction=True)
    
    # Split your data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(obss, actions, test_size=0.1, random_state=123)
    
    val_dataset = StateActionReturnDataset(X_train, args.context_length*3, y_train, done_idxs, rtgs, timesteps)
    print(f"vocab size: {val_dataset.vocab_size}")
    
    # Define the model here
    mconf = GPTConfig(val_dataset.vocab_size, val_dataset.block_size, n_layer=6, n_head=8, n_embd=128, model_type=args.model_type, max_timestep=max(timesteps), input_dim=len(obss[0]))
    model = GPT(mconf)

    # Train the model for a fixed number of epochs
    epochs = 10
    tconf = TrainerConfig(max_epochs=epochs, batch_size=args.batch_size, learning_rate=6e-4, lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*args.context_length*3, num_workers=4, seed=args.seed, model_type=args.model_type, env=args.env, max_timestep=max(timesteps))
    
    rtg = 100
    max_ep_len = 100
    trainer = Trainer(model, train_dataset, None, tconf, args.env, rtg=rtg, max_ep_len=max_ep_len, num_eval_episodes=1)
    
    trainer.train()
    val_loss, val_accuracy = trainer.evaluate_model()

    # Return the validation loss as the objective value to be minimized
    return val_loss


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
best_params = study.best_params
learning_rate = best_params['learning_rate']
num_hidden_units = best_params['num_hidden_units']
learning_rate, num_hidden_units
# # Train your final model using the best hyperparameters
# final_model = model(num_hidden_units)
# optimizer = optim.Adam(final_model.parameters(), lr=learning_rate)

# Train the model on the entire training data
# ...


In [ ]:
# Load the val_dataset
val_data_dir = "../atari/data-for-dt/smart-climate-val-trajectories-v2.pkl"
# Create the dataset first
obss, actions, returns, done_idxs, rtgs, timesteps = create_dataset(val_data_dir, total_trajectories=8000, apply_dim_reduction=True)
# Sanity check
# Are there any nan values in the obss
print(f"There are nan values in the obss: {np.isnan(np.array(obss)).any()}")
print("*" * len(args.env + "Environment"))
print(f"{args.env} Environment")
print("*" * len(args.env + "Environment"))
print(f"total obss: {len(obss)}\ntotal actions: {actions.shape}\ntotal returns: {returns.shape}\ntimesteps: {len(timesteps)}")

In [ ]:
val_dataset = StateActionReturnDataset(obss, args.context_length*3, actions, done_idxs, rtgs, timesteps)
print(f"vocab size: {train_dataset.vocab_size}")

In [ ]:
# initialize a trainer instance and kick off training
epochs = 10
tconf = TrainerConfig(max_epochs=epochs, batch_size=args.batch_size, learning_rate=6e-4, lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*args.context_length*3, num_workers=4, seed=args.seed, model_type=args.model_type, env=args.env, max_timestep=max(timesteps))

In [ ]:
rtg = 100
max_ep_len = 100
trainer = Trainer(model, train_dataset, None, tconf, args.env, rtg=rtg, max_ep_len=max_ep_len, num_eval_episodes=10)
# avg_return, predicted_actions, target_actions, epoch_losses = trainer.train()
# print(f"Average reward achieved: {avg_return:.2f} with RTG: {rtg}")